# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846849492074                   -0.70    4.8
  2   -7.852321733727       -2.26       -1.53    1.0
  3   -7.852614249310       -3.53       -2.56    1.2
  4   -7.852645892754       -4.50       -2.86    2.5
  5   -7.852646460671       -6.25       -3.13    1.0
  6   -7.852646678811       -6.66       -4.02    1.2
  7   -7.852646686613       -8.11       -5.06    2.0
  8   -7.852646686722       -9.96       -5.36    2.0
  9   -7.852646686728      -11.21       -5.62    1.0
 10   -7.852646686730      -11.68       -6.63    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846811066590                   -0.70           4.5
  2   -7.852538152961       -2.24       -1.63   0.80    2.2
  3   -7.852637751031       -4.00       -2.72   0.80    1.0
  4   -7.852646479027       -5.06       -3.35   0.80    2.2
  5   -7.852646676510       -6.70       -4.20   0.80    1.5
  6   -7.852646686577       -8.00       -4.70   0.80    2.0
  7   -7.852646686711       -9.87       -5.63   0.80    1.2
  8   -7.852646686729      -10.74       -6.05   0.80    2.2


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.370033e+01     3.308369e+00
 * time: 0.4555649757385254
     1     1.015241e+00     1.816783e+00
 * time: 0.6923859119415283
     2    -1.568989e+00     1.907528e+00
 * time: 0.7245888710021973
     3    -3.746339e+00     1.595216e+00
 * time: 0.7722969055175781
     4    -5.098216e+00     1.591197e+00
 * time: 0.8187978267669678
     5    -6.767144e+00     9.110874e-01
 * time: 0.8669188022613525
     6    -7.399833e+00     4.780487e-01
 * time: 0.9208719730377197
     7    -7.642623e+00     2.454088e-01
 * time: 0.9531338214874268
     8    -7.741830e+00     1.171936e-01
 * time: 0.9853618144989014
     9    -7.788832e+00     1.296599e-01
 * time: 1.017906904220581
    10    -7.816401e+00     1.010380e-01
 * time: 1.0500140190124512
    11    -7.833832e+00     6.586149e-02
 * time: 1.082502841949463
    12    -7.841420e+00     7.145352e-02
 * time: 1.1171059608459473
    13    -7.844433e+00     6.852982e-02
 * time: 1.149503946304

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846866662412                   -0.70    4.8


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645910579                   -1.64
  2   -7.852646686730       -6.11       -3.71
  3   -7.852646686730      -13.29       -7.25


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.72331835364057e-7
|ρ_newton - ρ_scfv| = 1.6977618814291134e-7
|ρ_newton - ρ_dm|   = 6.796344828107286e-10
